## GPU Demos

### Install and Imports

Do a `pip install` of the [numba](https://numba.pydata.org/) library and check for where the cuda `.so` files are kept.  If a `.so` appears CUDA is likely installed.

In [1]:
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!pip install pycuda

     |████████████████████████████████| 3.3 MB 3.2 MB/s 
  Using cached numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.3 MB)
     |████████████████████████████████| 26.3 MB 4.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.20.3 which is incompatible.
/var/lib/docker/overlay2/1c2ffe0fc1eae0471e8f1e481b3d5f7dfa423b8ac4e9ad95ae79ccf9a7e34847/diff/usr/local/cuda-10.0/nvvm/libdevice
/var/lib/docker/overlay2/a3c830f3b106269d4d3bf4296d6e398983a01f83781a801f1c6ef033fa9b943c/diff/usr/local/cuda-9.0/nvvm/libdevice
/var/lib/docker/overlay2/40828fa6b89d0389f20e9cd00ea3abf4ad0f80a27ea5473fef26d6155e15dfa9/diff/usr/local/cuda-10.2

Install libraries

In [2]:
!nvidia-smi

Tue Nov 30 03:28:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 16%   33C    P2    64W / 250W |   5864MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 16%   27C    P8    29W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np
from time import time

In [4]:
ker = """
  __device__ void swap(int & a, int & b){
    int tmp = a;
    a = b;
    b = tmp;
}

__global__ void bitonicSort(int * A, int N){
    extern __shared__ int shared[];
		int tid = threadIdx.x + blockDim.x * blockIdx.x;
    // Copy input to shared mem.
    shared[tid] = A[tid];
    __syncthreads();
    // Parallel bitonic sort.
    for (int k = 2; k <= N; k *= 2){
        // Bitonic merge:
        for (int j = k / 2; j>0; j /= 2){
            int ixj = tid ^ j;
            if (ixj > tid){
                if ((tid & k) == 0){
                    if (shared[tid] > shared[ixj]){
                        swap(shared[tid], shared[ixj]);
                    }
                }
                else{
                    if (shared[tid] < shared[ixj]){
                        swap(shared[tid], shared[ixj]);
                    }
                }
            }
            __syncthreads();
        }
    }
    // Write result.
    A[tid] = shared[tid];
}
"""

In [5]:
N = 512 #lenght of A
A = np.int32(np.random.randint(1, 20, N)) #random numbers in A
BLOCK_SIZE = N
NUM_BLOCKS = (N + BLOCK_SIZE-1)//BLOCK_SIZE
mod = SourceModule(ker)
bitonicSort = mod.get_function("bitonicSort")
t1 = time()
bitonicSort(drv.InOut(A), np.int32(N), block=(BLOCK_SIZE,1,1), grid=(NUM_BLOCKS,1), shared=4*N)
t2 = time()
print("Execution Time {0}".format(t2 - t1))
print(A)

Execution Time 0.0029528141021728516
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 13 13 13 13 1